# Single Type Experiment

In [1]:
import warnings;
warnings.filterwarnings('ignore');

from time import sleep
from tqdm.auto import tqdm

import sys
import importlib
import numpy as np
import nbformat
# import plotly.express
# import plotly.express as px
import pandas as pd
import scipy.optimize as optimization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def offline_opt(budget, size):
    # return [budget / np.sum(size) * size[i] for i in range(len(size))]
    return [budget / np.sum(size) for i in range(len(size))]

In [3]:
# def hope_online(budget, size, mean):
#     # initialize allocation vector
#     allocation = np.zeros(len(size))
#     budget_remaining = budget

#     c = 1

#     conf_bnd = c*np.sqrt(np.mean(mean)*len(size)*np.log(len(size)))

#     thresh = budget / (size[0] + np.sum(mean[1:]) + np.sqrt(np.log(len(size)-1) * (len(size)-1)))

#     for i in range(len(allocation)):
#        # allocation[i] = min(budget_remaining, thresh * size[i])
#         allocation[i] = min(budget_remaining / size[i], thresh)
#         budget_remaining -= allocation[i] * size[i]
#     return allocation

In [4]:
def hope_full(budget, size, mean):
    allocation = np.zeros(len(size))
    budget_remaining = budget
    
    c = 1
    conf_bnd = (np.sum(mean[1:]))**(0.6) #c*np.sqrt(np.sum(mean)* np.log(len(size)-1)) 

    thresh_lower = budget / (size[0] + np.sum(mean[1:]) + conf_bnd)
    thresh_upper = budget / (size[0] + np.sum(mean[1:]) - conf_bnd)

    for i in range(len(allocation)-1):
        # TODO: add on confidence terms for (T - i) people here
        #rem = len(allocation) - (i + 1)
        budget_upper = thresh_upper*(size[i] + np.sum(mean[i+1:])) # + c*np.sqrt(rem * np.log(rem)))
        budget_lower = thresh_lower*(size[i] + np.sum(mean[i+1:])) # + c*np.sqrt(rem * np.log(rem)))

        gam = (budget_remaining - budget_lower) / (budget_upper - budget_lower)
        
        if gam >= 0.95: allocation[i] = thresh_upper
        else: allocation[i] = thresh_lower
        
        if allocation[i] * size[i] <= budget_remaining:
            budget_remaining -= allocation[i] * size[i]
        else:
            allocation[i] = budget_remaining/size[i]
            budget_remaining = 0

        if budget_remaining < 0:
            print("Error: Negative Budget")

            
    # fix for last agent
    allocation[-1] = min(thresh_upper, budget_remaining / size[-1])
    
    return allocation



In [5]:

####### FIXED DEFINITION OF FIXED THRESHOLD WITH NO ADAPTIVE PARTS
# \sqrt{T} waste, \sqrt{T} L1, 1 / \sqrt{T} for Linf

# def hope_threshold(budget, size, mean):
#     allocation = np.zeros(len(size))
#     budget_remaining = budget
   
#     c = 1
    
#     conf_bnd = c*np.sqrt(np.mean(mean)*len(size)*np.log(len(size)))
    
#     thresh = budget / (size[0] + np.sum(mean[1:]) + np.sqrt(np.log(len(size)-1) * (len(size)-1)))

#     for i in range(len(allocation)):
#        # allocation[i] = min(budget_remaining, thresh * size[i])
#         allocation[i] = min(budget_remaining / size[i], thresh)
#         budget_remaining -= allocation[i] * size[i]
#     return allocation


##### MODIFIED VERSION WHICH IS GREEDY WHEN POSSIBLE
#


def hope_threshold(budget, size, mean):
    allocation = np.zeros(len(size))
    budget_remaining = budget
    
    
    
    c = 0.7

    rem = len(allocation)
    conf_bnd = c*np.sqrt(np.mean(mean)*(rem* np.log(rem))) 

    # calculates initial thresholds
    thresh_lower = budget / (np.sum(size[0]) + np.sum(mean[1:]) + conf_bnd)
    thresh_upper = budget / (np.sum(size[0]) + np.sum(mean[1:]) - conf_bnd)  
    # print(thresh_lower, thresh_upper)
    
    for i in range(len(allocation)):
    
    
        rem = len(allocation) - i
        conf_bnd = c*np.sqrt(np.mean(mean)*(rem* np.log(rem))) 
    
        
        budget_upper = thresh_lower * (np.sum(mean[i+1:]) + conf_bnd)
        
        
        
        if budget_remaining / size[i] < thresh_lower:
            # print(str(i) + ' giving rest of budget!')
            allocation[i] = budget_remaining / size[i]
        
        elif budget_remaining - size[i] * thresh_upper >= budget_upper:
            allocation[i] = thresh_upper

        else:
            allocation[i] = thresh_lower

            
        budget_remaining -= allocation[i] * size[i]

        
    if np.round(budget_remaining, 3) < 0:
        print(budget_remaining)
        print('Error: Negative Budget')
                
        
    return allocation



In [6]:
def hope_modified(budget, size, mean):
    allocation = np.zeros(len(size))
    budget_remaining = budget
    
    
    
    c = 0.7

    rem = len(allocation)
    conf_bnd = c*np.sqrt(np.mean(mean)*(rem* np.log(rem))) 


    # print(thresh_lower, thresh_upper)
    
    for i in range(len(allocation)):
    
    
        rem = len(allocation) - i
        conf_bnd = c*np.sqrt(np.mean(mean)*(rem* np.log(rem))) 
        # calculates initial thresholds
        thresh_lower = budget / (np.sum(size[0:i+1]) + np.sum(mean[i+1:]) + conf_bnd)
        thresh_upper = budget / (np.sum(size[0:i+1]) + np.sum(mean[i+1:]) - conf_bnd)      
        
        budget_upper = thresh_lower * (np.sum(mean[i+1:]) + conf_bnd)
        
        
        
        if budget_remaining / size[i] < thresh_lower:
            # print(str(i) + ' giving rest of budget!')
            allocation[i] = budget_remaining / size[i]
        
        elif budget_remaining - size[i] * thresh_upper >= budget_upper:
            allocation[i] = thresh_upper

        else:
            allocation[i] = thresh_lower

            
        budget_remaining -= allocation[i] * size[i]

        
    if np.round(budget_remaining, 3) < 0:
        print(budget_remaining)
        print('Error: Negative Budget')
                
        
    return allocation

### Test

In [7]:
n = 6
sorted_distribution = np.asarray([np.arange(2) + 1 for x in range(n)])
weights = np.asarray([np.zeros(2)+0.5 for x in range(n)])
mean = [np.dot(weights[i], sorted_distribution[i]) for i in range(n)]

expected_demand = 1.5
budget = n * expected_demand
print(budget)

9.0


In [8]:
print(weights)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


In [9]:
print(sorted_distribution)

[[1 2]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [1 2]]


In [10]:
size = [2., 1., 1., 1., 2., 1.]

In [11]:
len(size)


6

In [12]:
print(offline_opt(budget, size))

[1.125, 1.125, 1.125, 1.125, 1.125, 1.125]


In [13]:
print(hope_online(budget, size, mean))

NameError: name 'hope_online' is not defined

In [ ]:
print(hope_full(budget, size, mean))

In [ ]:
print(hope_modified(budget, size, mean))

In [ ]:
print(np.sum(hope_threshold(budget, size, mean)))

### Scaling with n

In [ ]:
#np.random.choice(size=5, a=[1.,2.], p=[1/2, 1/2])
1+np.random.poisson(size=5,lam = 1.5)

In [ ]:
data_dict = {'NumGroups':[], 'Norm':[], 'Fixed_Threshold':[], 'Hope_Sid':[], 'Adapt_Threshold':[], 'Adapt_Init_Threshold':[]}

num_iterations = 20
max_n = 100

mean_size = 1.5

for n in tqdm(np.arange(2,max_n+1)):
    expected_size = np.zeros(n) + mean_size
    budget = n
    
    for i in range(num_iterations):
        
        data_dict['NumGroups'].append(n)
        
        size = np.random.choice(size=n, a=[1.,2.], p=[1/2, 1/2])
        # size = 1+np.random.poisson(size=n,lam = mean_size-1)
        opt = offline_opt(budget, size)
        hope_on = hope_online(budget, size, expected_size)
        hope_ful = hope_full(budget, size, expected_size)
        hope_mod = hope_modified(budget, size, expected_size)
        hope_thresh = hope_threshold(budget, size, expected_size)
     
        data_dict['Norm'].append('L1')
        data_dict['Fixed_Threshold'].append(np.sum(np.absolute(opt - hope_on)))
        data_dict['Hope_Sid'].append(np.sum(np.absolute(opt-hope_ful)))
        data_dict['Adapt_Threshold'].append(np.sum(np.absolute(opt-hope_mod)))
        data_dict['Adapt_Init_Threshold'].append(np.sum(np.absolute(opt-hope_thresh)))
        # data_dict['Waste_Calc'].append(0)

        data_dict['NumGroups'].append(n)
        data_dict['Norm'].append('Linf')
        data_dict['Fixed_Threshold'].append(np.max(np.absolute(opt - hope_on)))
        data_dict['Hope_Sid'].append(np.max(np.absolute(opt-hope_ful)))
        data_dict['Adapt_Threshold'].append(np.max(np.absolute(opt-hope_mod)))
        data_dict['Adapt_Init_Threshold'].append(np.max(np.absolute(opt-hope_thresh)))
        # data_dict['Waste_Calc'].append(0)
        
        data_dict['NumGroups'].append(n)
        data_dict['Norm'].append('Waste')
        data_dict['Fixed_Threshold'].append(budget - np.dot(hope_on, size))
        data_dict['Hope_Sid'].append(budget - np.dot(hope_ful,size))
        data_dict['Adapt_Threshold'].append(budget - np.dot(hope_mod, size))
        data_dict['Adapt_Init_Threshold'].append(budget - np.dot(hope_thresh, size))
        
        rem = n
        c = 0.7
        conf_bnd = c*np.sqrt(np.mean(mean)*(rem* np.log(rem))) 

        # calculates initial thresholds
        thresh_lower = budget / (np.sum(size[0]) + np.sum(mean[1:]) + conf_bnd)
        thresh_upper = budget / (np.sum(size[0]) + np.sum(mean[1:]) - conf_bnd)  

    
        # data_dict['Waste_Calc'].append(min(0, budget - np.sum(size)*thresh_lower))
    sleep(0)

In [ ]:
df = pd.DataFrame(data_dict).melt(id_vars=["NumGroups", 'Norm'])
# df.to_csv('scale_with_n.csv')
df.columns


#filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
#plt.figure(figsize=(20,10))
#plt.title('Simple Distribution L1')
#sns.lineplot(x='NumGroups', y='value', hue='variable', data=df[df.Norm == 'L1'])

filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
plt.figure(figsize=(16,8))
plt.title('Simple Distribution LInf')
sns.lineplot(x='NumGroups', y='value', hue='variable', data=df[df.Norm == 'Linf'], ci=None)


filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
plt.figure(figsize=(16,8))
plt.title('Simple Distribution Waste')
sns.lineplot(x='NumGroups', y='value', hue='variable', data=df[df.Norm == 'Waste'], ci=None)


### Group by Group Difference

In [ ]:
n = 100
num_iterations = 100

mean_size = 1.5
expected_size = np.zeros(n) + mean_size
group = np.arange(n)
budget = n
group = np.arange(n)

score_hope_online = np.zeros((n,num_iterations))
score_hope_full = np.zeros((n,num_iterations))
score_hope_modified = np.zeros((n, num_iterations))
score_hope_threshold = np.zeros((n, num_iterations))

for i in tqdm(range(num_iterations)):
    # size = 1. + np.random.poisson(size=n,lam = mean_size-1)
    size = np.random.choice(size=n, a=[1.,2.], p=[1/2, 1/2])
    opt = offline_opt(budget, size)
    hope_on = hope_online(budget, size, expected_size)
    hope_ful = hope_full(budget, size, expected_size)
    hope_mod = hope_modified(budget, size, expected_size)
    hope_thresh = hope_threshold(budget, size, expected_size)
    
    # comparing hope_online
    
    score_hope_online[:,i]    = opt - hope_on
    score_hope_full[:,i]      = opt - hope_ful
    score_hope_modified[:,i]  = opt - hope_mod
    score_hope_threshold[:,i] = opt - hope_thresh
    sleep(0)

In [ ]:
size

In [ ]:
np.max(np.abs(score_hope_online), axis=0).shape

In [ ]:
data_dict = {'Agent':group, 'Zero': np.zeros(n), 'Fixed_Threshold': np.average(score_hope_online, axis=1), 'Hope_Sid':np.average(score_hope_full, axis=1), 'Adap_Threshold':np.average(score_hope_modified, axis=1), 'Adapt_Init_Threshold':np.average(score_hope_threshold, axis=1)}
df_uniform = pd.DataFrame(data_dict).melt(id_vars="Agent")

plt.figure(figsize=(20,10))
sns.lineplot(x='Agent', y='value', hue = 'variable', data=df_uniform)
plt.title('Allocation Difference')
plt.xlabel('Value')
plt.plot()

Note: Discrepancy is between where max and average are between this plot and the other one.

In [ ]:
d = {'Fixed_Threshold': np.average(np.max(np.abs(score_hope_online), axis=0)), 'Hope_Sid':np.average(np.max(np.abs(score_hope_full), axis=0)), 'Adap_Threshold':np.average(np.max(np.abs(score_hope_modified), axis=0)), 'Adapt_Init_Threshold':np.average(np.max(np.abs(score_hope_threshold), axis=0))}

In [ ]:
d